# Deploying FastAPI Applications with Kubernetes

In this tutorial, we'll explore how to deploy **FastAPI** applications using **Kubernetes**, an open-source container orchestration system for automating deployment, scaling, and management of containerized applications. We'll cover the fundamental concepts of Kubernetes, how to containerize your FastAPI application with Docker, and demonstrate how to deploy, manage, and scale your application using Kubernetes.

## Table of Contents

1. [Introduction](#1-introduction)
2. [Prerequisites](#2-prerequisites)
3. [Understanding Kubernetes](#3-understanding-kubernetes)
   - [3.1. What is Kubernetes?](#31-what-is-kubernetes)
   - [3.2. Key Kubernetes Concepts](#32-key-kubernetes-concepts)
4. [Setting Up the Environment](#4-setting-up-the-environment)
   - [4.1. Installing Docker](#41-installing-docker)
   - [4.2. Installing Kubernetes](#42-installing-kubernetes)
   - [4.3. Setting Up Minikube](#43-setting-up-minikube)
5. [Containerizing the FastAPI Application](#5-containerizing-the-fastapi-application)
   - [5.1. Creating a Simple FastAPI Application](#51-creating-a-simple-fastapi-application)
   - [5.2. Writing a Dockerfile](#52-writing-a-dockerfile)
   - [5.3. Building the Docker Image](#53-building-the-docker-image)
6. [Deploying to Kubernetes](#6-deploying-to-kubernetes)
   - [6.1. Creating a Kubernetes Deployment](#61-creating-a-kubernetes-deployment)
   - [6.2. Creating a Kubernetes Service](#62-creating-a-kubernetes-service)
   - [6.3. Applying the Configuration](#63-applying-the-configuration)
7. [Exposing the Application](#7-exposing-the-application)
   - [7.1. Using NodePort Service](#71-using-nodeport-service)
   - [7.2. Using Ingress Controller](#72-using-ingress-controller)
8. [Scaling the Application](#8-scaling-the-application)
   - [8.1. Manual Scaling](#81-manual-scaling)
   - [8.2. Autoscaling with HPA](#82-autoscaling-with-hpa)
9. [Updating the Application](#9-updating-the-application)
   - [9.1. Rolling Updates](#91-rolling-updates)
   - [9.2. Rollbacks](#92-rollbacks)
10. [ConfigMaps and Secrets](#10-configmaps-and-secrets)
    - [10.1. Using ConfigMaps](#101-using-configmaps)
    - [10.2. Using Secrets](#102-using-secrets)
11. [Persistent Storage with Volumes](#11-persistent-storage-with-volumes)
12. [Logging and Monitoring](#12-logging-and-monitoring)
13. [Conclusion](#13-conclusion)
14. [References](#14-references)

## 1. Introduction

Kubernetes has become the de facto standard for deploying and managing containerized applications at scale. By leveraging Kubernetes, you can automate deployment, scaling, and operations of application containers across clusters of hosts.

In this tutorial, we'll:

- Understand the basic concepts of Kubernetes.
- Containerize a FastAPI application using Docker.
- Deploy the application to a Kubernetes cluster.
- Explore scaling, updating, and managing the application using Kubernetes features.

## 2. Prerequisites

Before we begin, ensure you have the following:

- **Python 3.7+** installed.
- Basic knowledge of **Python**, **FastAPI**, and **Docker**.
- Familiarity with command-line operations.
- **Docker** and **Kubernetes** installed on your machine.


## 3. Understanding Kubernetes

### 3.1. What is Kubernetes?

**Kubernetes** (often abbreviated as **K8s**) is an open-source platform designed to automate deploying, scaling, and operating containerized applications. It groups containers that make up an application into logical units for easy management and discovery.

### 3.2. Key Kubernetes Concepts

1. Core Concepts

- **Cluster**: A set of worker machines, called nodes, that run containerized applications. It's the foundation of Kubernetes, providing a unified platform for deploying and managing containers.

- **Node**: A single machine in a cluster. It can be a physical or virtual machine, responsible for running pods. Nodes are managed by the control plane.

- **Pod**: The smallest deployable unit in Kubernetes, which can contain one or more containers. Pods are ephemeral and can be created, destroyed, and rescheduled based on the cluster's needs.

Example Pod YAML with multiple containers:

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: web-db-pod
spec:
  containers:
  - name: web-server
    image: nginx:1.14.2
    ports:
    - containerPort: 80
  - name: database
    image: mysql:5.7
    ports:
    - containerPort: 3306
    env:
    - name: MYSQL_ROOT_PASSWORD
      value: example-password
```
This example demonstrates a pod with two containers: a web server (nginx) and a database (MySQL).


2. Workload Management

- **Deployment**: Manages stateless applications; provides updates and scaling. It ensures a specified number of pod replicas are running at any given time.
  Example: A deployment that maintains three replicas of a web application pod.

- **ReplicaSet**: Ensures that a specified number of pod replicas are running at any given time. It's often used indirectly through Deployments.
  Example: A ReplicaSet maintaining 3 identical pods of a web application.

- **StatefulSet**: Manages stateful applications, providing guarantees about the ordering and uniqueness of pods.
  Example: A StatefulSet managing a distributed database where each pod has a unique, persistent identifier.

- **DaemonSet**: Ensures that all (or some) nodes run a copy of a pod. Useful for cluster-wide services.
  Example: A DaemonSet running a log collection agent on every node in the cluster.

- **Job**: Creates one or more pods and ensures that a specified number of them successfully terminate.
  Example: A Job that processes a batch of data and then terminates.

- **CronJob**: Manages time-based Jobs, creating Jobs on a schedule.
  Example: A CronJob that runs a backup operation every night at midnight.

Example Deployment YAML:
```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:1.14.2
        ports:
        - containerPort: 80
```

3. Networking and Service Discovery

- **Service**: An abstraction which defines a logical set of Pods and a policy by which to access them. It provides a stable network endpoint for pods.
  Example: A service that load balances traffic across multiple web application pods.

- **Ingress**: Exposes HTTP and HTTPS routes from outside the cluster to services within the cluster. It acts as a smart router for your cluster.
  Example: An ingress that routes traffic to different services based on the URL path.

- **NetworkPolicy**: A specification of how groups of pods are allowed to communicate with each other and other network endpoints.
  Example: A NetworkPolicy that allows database pods to accept connections only from backend pods.

Example Service and Ingress YAML:
```yaml
---
apiVersion: v1
kind: Service
metadata:
  name: nginx-service
spec:
  selector:
    app: nginx
  ports:
    - protocol: TCP
      port: 80
      targetPort: 80
---
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: nginx-ingress
spec:
  rules:
  - host: myapp.example.com
    http:
      paths:
      - path: /
        pathType: Prefix
        backend:
          service:
            name: nginx-service
            port: 
              number: 80
```

4. Storage

- **Volume**: Used to provide storage for applications running in a Pod. Volumes can be ephemeral or persistent.
  Example: A volume that stores temporary data for a pod.

- **PersistentVolume (PV)**: A piece of storage in the cluster that has been provisioned by an administrator or dynamically provisioned using Storage Classes. It exists independently of any pod.
  Example: A PV representing a 100GB block storage volume.

- **PersistentVolumeClaim (PVC)**: A request for storage by a user. It's similar to a pod in that it consumes PV resources.
  Example: A PVC requesting 10GB of storage for a database.

Example PV and PVC YAML:
```yaml
---
apiVersion: v1
kind: PersistentVolume
metadata:
  name: pv-example
spec:
  capacity:
    storage: 1Gi
  accessModes:
    - ReadWriteOnce
  persistentVolumeReclaimPolicy: Retain
  storageClassName: standard
  nfs:
    server: nfs-server.default.svc.cluster.local
    path: "/path/to/data"
---
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: pvc-example
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 1Gi
  storageClassName: standard
```

5. Configuration and Secrets

- **ConfigMap**: Used to store non-confidential configuration data in key-value pairs. It allows you to decouple configuration from container images.
  Example: A ConfigMap storing database connection strings or feature flags.

- **Secret**: Used to store sensitive information like passwords, tokens, and keys. Similar to ConfigMaps, but specifically for confidential data.
  Example: A Secret storing database passwords or API keys.

Example ConfigMap and Secret YAML:
```yaml
---
apiVersion: v1
kind: ConfigMap
metadata:
  name: app-config
data:
  APP_COLOR: blue
  APP_MODE: prod
---
apiVersion: v1
kind: Secret
metadata:
  name: app-secret
type: Opaque
data:
  DB_Password: cGFzc3dvcmQxMjM=  # base64 encoded "password123"
```

6. Scheduling and Scaling

- **Namespace**: A mechanism for isolating groups of resources within a single cluster. It helps in organizing and managing different environments or projects.
  Example: Separate namespaces for development, staging, and production environments.

- **Horizontal Pod Autoscaler (HPA)**: Automatically scales the number of pods in a deployment or replica set based on observed CPU utilization or other metrics.
  Example: An HPA that increases the number of web server pods when CPU usage exceeds 80%.

- **Vertical Pod Autoscaler (VPA)**: Automatically adjusts the CPU and memory reservations for pods to better match their needs.
  Example: A VPA that increases the memory limit for a pod that consistently uses more memory than initially allocated.

Example Namespace and HPA YAML:
```yaml
---
apiVersion: v1
kind: Namespace
metadata:
  name: development
---
apiVersion: autoscaling/v2beta1
kind: HorizontalPodAutoscaler
metadata:
  name: nginx-hpa
spec:
  scaleTargetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: nginx-deployment
  minReplicas: 1
  maxReplicas: 10
  metrics:
  - type: Resource
    resource:
      name: cpu
      targetAverageUtilization: 50
```

7. Security and Access Control

- **ServiceAccount**: Provides an identity for processes that run in a pod, used to control access to the Kubernetes API.
  Example: A ServiceAccount that allows a monitoring pod to read metrics from other pods in the cluster.

Example ServiceAccount and NetworkPolicy YAML:
```yaml
---
apiVersion: v1
kind: ServiceAccount
metadata:
  name: app-service-account
---
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: allow-nginx-ingress
spec:
  podSelector:
    matchLabels:
      app: nginx
  ingress:
  - from:
    - podSelector:
        matchLabels:
          role: frontend
    ports:
    - protocol: TCP
      port: 80
```

8. Extending Kubernetes

- **CustomResourceDefinition (CRD)**: Extends the Kubernetes API to create custom resources tailored to your specific application needs.
  Example: A CRD that defines a new "Database" resource type with fields specific to your database management needs.

- **Operator**: A method of packaging, deploying, and managing a Kubernetes application using custom resources and controllers.
  Example: A PostgreSQL operator that automates database provisioning, backup, and scaling.

- **Helm**: A package manager for Kubernetes that allows you to define, install, and upgrade complex Kubernetes applications.
  Example: Using Helm to install a pre-configured monitoring stack like Prometheus and Grafana.

Example CRD YAML:
```yaml
apiVersion: apiextensions.k8s.io/v1
kind: CustomResourceDefinition
metadata:
  name: backups.myapp.example.com
spec:
  group: myapp.example.com
  versions:
    - name: v1
      served: true
      storage: true
      schema:
        openAPIV3Schema:
          type: object
          properties:
            spec:
              type: object
              properties:
                frequency:
                  type: string
                destination:
                  type: string
  scope: Namespaced
  names:
    plural: backups
    singular: backup
    kind: Backup
    shortNames:
    - bkp
```

## 4. Setting Up the Environment

### 4.1. Installing Docker

If you don't have Docker installed, download and install it from the [official website](https://www.docker.com/get-started).

Verify the installation:

```bash
docker --version
```

### 4.2. Installing Kubernetes

For local development, you can use **Minikube** or **Docker Desktop**, which comes with Kubernetes support.

### 4.3. Setting Up Minikube

**Minikube** is a tool that runs a single-node Kubernetes cluster on your local machine.

#### Install Minikube

- **macOS**:

  ```bash
  brew install minikube
  ```

- **Linux**:

  ```bash
  curl -Lo minikube https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64
  chmod +x minikube
  sudo mv minikube /usr/local/bin/
  ```

- **Windows**:

  Download the installer from the [official site](https://minikube.sigs.k8s.io/docs/start/).

#### Start Minikube

```bash
minikube start
```

Verify the cluster is running:

```bash
kubectl cluster-info
```


## 5. Containerizing the FastAPI Application

### 5.1. Creating a Simple FastAPI Application

Create a new project directory:

```bash
mkdir fastapi-kubernetes
cd fastapi-kubernetes
```

Create a virtual environment and activate it:

```bash
python -m venv venv
source venv/bin/activate  # Windows: venv\Scripts\activate
```

Install FastAPI and Uvicorn:

```bash
pip install fastapi uvicorn
```

Create `main.py`:

```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def read_root():
    return {"message": "Hello, Kubernetes!"}

@app.get("/items/{item_id}")
async def read_item(item_id: int):
    return {"item_id": item_id}
```

Test the application locally:

```bash
uvicorn main:app --reload
```

Visit `http://localhost:8000` to see the output.

### 5.2. Writing a Dockerfile

Create a file named `Dockerfile` in the project directory:

```dockerfile
# Use the official Python image as the base image
FROM python:3.12-slim

# Working directory is the directory where the application will be run in the container
WORKDIR /app

# Copy the requirements file
COPY requirements.txt .

# Install the dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy the application code
COPY . .

# Expose the port
EXPOSE 80

# Command to run the application
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "80"]
```

Create `requirements.txt`:

```text
fastapi
uvicorn[standard]
```

### 5.3. Building the Docker Image

Build the Docker image:

```bash
docker build -t fastapi-kubernetes:latest .
```

Verify the image is built:

```bash
docker images
```

Run the Docker container:

```bash
docker run -d --name fastapi-app -p 8000:80 fastapi-kubernetes:latest
```

Visit `http://localhost:8000` to confirm it's working.

## 6. Deploying to Kubernetes

### 6.1. Creating a Kubernetes Deployment

Create a file named `deployment.yaml`:

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: fastapi-deployment
spec:
  replicas: 2
  selector:
    matchLabels:
      app: fastapi
  template:
    metadata:
      labels:
        app: fastapi
    spec:
      containers:
      - name: fastapi
        image: fastapi-kubernetes:latest
        ports:
        - containerPort: 80
```

### 6.2. Creating a Kubernetes Service

Create a file named `service.yaml`:

```yaml
apiVersion: v1
kind: Service
metadata:
  name: fastapi-service
spec:
  selector:
    app: fastapi
  type: NodePort
  ports:
    - protocol: TCP
      port: 80
      targetPort: 80
      nodePort: 30001  # NodePort range is typically 30000-32767
```

### 6.3. Applying the Configuration

Before applying the configuration, we need to ensure Kubernetes can access our Docker image.

#### Option 1: Use Minikube's Docker Daemon

Set your shell to use the Minikube Docker daemon:

```bash
eval $(minikube docker-env)
```

Now, build the Docker image again, and it will be available to Minikube's Docker daemon:

```bash
docker build -t fastapi-kubernetes:latest .
```

#### Option 2: Push to Docker Hub (Skip if using Option 1)

Tag the image:

```bash
docker tag fastapi-kubernetes:latest your-dockerhub-username/fastapi-kubernetes:latest
```

Push the image:

```bash
docker push your-dockerhub-username/fastapi-kubernetes:latest
```

Update `deployment.yaml` to use the Docker Hub image:

```yaml
        image: your-dockerhub-username/fastapi-kubernetes:latest
```

#### Apply the configurations:

```bash
kubectl apply -f deployment.yaml
kubectl apply -f service.yaml
```

Check the status:

```bash
kubectl get deployments
kubectl get pods
kubectl get services
```

## 7. Exposing the Application

### 7.1. Using NodePort Service

Since we used `type: NodePort`, we can access the service via the node's IP address and the node port.

Get the Minikube IP:

```bash
minikube ip
```

Access the application:

```
http://<minikube-ip>:30001
```

### 7.2. Using Ingress Controller

For production environments, it's common to use an Ingress controller to manage external access.

#### Enable Ingress in Minikube:

```bash
minikube addons enable ingress
```

Create an `ingress.yaml`:

```yaml
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: fastapi-ingress
spec:
  rules:
  - host: fastapi.local
    http:
      paths:
      - path: /
        pathType: Prefix
        backend:
          service:
            name: fastapi-service
            port:
              number: 80
```

Add `fastapi.local` to your `/etc/hosts`:

```
127.0.0.1 fastapi.local
```

Apply the ingress configuration:

```bash
kubectl apply -f ingress.yaml
```

Access the application:

```
http://fastapi.local
```

## 8. Scaling the Application

### 8.1. Manual Scaling

Scale the deployment to 5 replicas:

```bash
kubectl scale deployment fastapi-deployment --replicas=5
```

Verify the pods:

```bash
kubectl get pods
```

### 8.2. Autoscaling with HPA

Use Kubernetes Horizontal Pod Autoscaler (HPA) to scale based on CPU usage.

First, enable metrics-server in Minikube:

```bash
minikube addons enable metrics-server
```

Create an HPA configuration:

```bash
kubectl autoscale deployment fastapi-deployment --cpu-percent=50 --min=2 --max=10
```

Check HPA status:

```bash
kubectl get hpa
```

To test scaling, you can generate load on the application.

## 9. Updating the Application

### 9.1. Rolling Updates

Make changes to your application code, for example, update the message in `main.py`:

```python
return {"message": "Hello, Kubernetes! Updated version."}
```

Rebuild the Docker image:

```bash
docker build -t fastapi-kubernetes:latest .
```

Apply the deployment again:

```bash
kubectl apply -f deployment.yaml
```

Kubernetes will perform a rolling update, gradually replacing old pods with new ones.

### 9.2. Rollbacks

If the update causes issues, you can rollback:

```bash
kubectl rollout undo deployment/fastapi-deployment
```

## 10. ConfigMaps and Secrets

### 10.1. Using ConfigMaps

Use ConfigMaps to decouple configuration artifacts from image content.

Create a `configmap.yaml`:

```yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: fastapi-config
data:
  MESSAGE: "Hello from ConfigMap!"
```

Update `deployment.yaml` to use the ConfigMap:

```yaml
        env:
        - name: MESSAGE
          valueFrom:
            configMapKeyRef:
              name: fastapi-config
              key: MESSAGE
```

Update `main.py` to use the environment variable:

```python
import os

@app.get("/")
async def read_root():
    message = os.getenv("MESSAGE", "Hello, Kubernetes!")
    return {"message": message}
```

### 10.2. Using Secrets

Use Secrets to manage sensitive information.

Create a secret:

```bash
kubectl create secret generic db-password --from-literal=PASSWORD=mysecretpassword
```

Update `deployment.yaml` to use the Secret:

```yaml
        env:
        - name: DB_PASSWORD
          valueFrom:
            secretKeyRef:
              name: db-password
              key: PASSWORD
```

## 11. Persistent Storage with Volumes

If your application requires persistent storage, you can use Kubernetes Volumes.

Example:

```yaml
        volumeMounts:
        - name: data-volume
          mountPath: /data
      volumes:
      - name: data-volume
        hostPath:
          path: /path/on/host
```

## 12. Logging and Monitoring

Set up logging and monitoring using tools like **Prometheus** and **Grafana**.

- **Prometheus**: For metrics collection.
- **Grafana**: For visualization.

Enable the Prometheus addon in Minikube:

```bash
minikube addons enable prometheus
```

You can also use **kubectl logs** to view logs:

```bash
kubectl logs <pod-name>
```

## 13. Conclusion

In this tutorial, we've:

- Understood the basics of Kubernetes and its key components.
- Containerized a FastAPI application using Docker.
- Deployed the application to a Kubernetes cluster.
- Explored how to expose, scale, update, and manage the application using Kubernetes features.
- Learned how to use ConfigMaps and Secrets for configuration management.

By leveraging Kubernetes, you can ensure your applications are scalable, resilient, and easier to manage in production environments.

## 14. References

- [Kubernetes Official Documentation](https://kubernetes.io/docs/home/)
- [FastAPI Documentation](https://fastapi.tiangolo.com/)
- [Docker Documentation](https://docs.docker.com/)
- [Minikube Documentation](https://minikube.sigs.k8s.io/docs/start/)
- [Kubernetes ConfigMaps](https://kubernetes.io/docs/concepts/configuration/configmap/)
- [Kubernetes Secrets](https://kubernetes.io/docs/concepts/configuration/secret/)
- [Kubernetes Ingress](https://kubernetes.io/docs/concepts/services-networking/ingress/)
- [Kubernetes Autoscaling](https://kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale/)